## PIL module documentation

The PIL module contains a polarity inversion line detector class and a parallelized implementation of the detector class. The detector class can be applied to image data or video data, and output a selective set of polarity inversion line for the image or the video.


### PIL detector class

_class_ PIL.__PIL__(polar_threshold=0.90,window_size=5,edge_retain=0.003,radius=12,min_samples=5)

Apply a Polarity Inversion Line(PIL) detection statistical algorithm over 2D vertical magnetic field image data. The algorithm consists of finding strong polar magnitude threshold, filtering pixels with local strong positive and negative polars, scharr edge detection and DBSCAN pixels clustering. Given an image, the algorithm is able to locate, at user's discretion, a certain number of Polarity Inversion Line segments.

Read more in the PIL module introduction notebook.
   

__Parameters__:

__polar__\_ __threshold__:  ___float, optional, default: 0.90, ranged from 0 to 1___

The quantile threshold for a pixel with positive/negative vertical magnetic field to be considered as a "strong" pixel. All positive/negative pixels of the image are ranked in an ascending order based on the magnitude of the vertical magnetic field. And only pixels lying on the top quantile will be considered as "strong" pixels. Setting the threshold close to 1 means only defining the very top few pixels with the largest vertical magnetic field component. 

__window__\_ __size__: ___Positive Int, optional, default: 5, typically odd___

The size of an arbitrary pixel's neighborhood. For each pixel, its neighborhood is defined as a square shape subimage of the original image that has the pixel of interest locating at the center of the subimage. Window_size is the length of the side of the subimage. This subimage includes several adjacent pixels which are considered as the "neighbors" of the pixel. Only pixels with "strong" positive and negative pixels in its neighborhood will be considered as candidates for polarity inversion line.


__edge__\_ __retain__: ___float, optional, default: 0.003, ranged from 0 to 1___
    
The fraction of high gradient pixels to retain for drawing the polarity inversion line. After filtering the pixels with local strong positive/negative neighbors, a scharr edge detection operator is applied to the filtered image, and each pixel has its gradient calculated. Edge_retain specifies the fraction of pixels with the highest gradients to retain for finally drawing the polarity inversion line.

__radius__: ___Int, optional, default: 12, positive___

The maximum distance for two pixels to be considered as neighbors of each other on the same polarity inversion line segment. After edge detection, the algorithm finally filters a set of pixels to draw the polarity inversion line. For two points to be considered as consisting of the same PIL, they must be reachable from each other in the sense that there is a path of core points along which any two points on the path has a distance less than radius. See _sklearn.cluster.DBSCAN_ __eps__ parameter for detail.

__min_samples__: ___Int, optional, default: 5, positive___

The number of pixels in a neighborhood for a pixel to be considered as a core point in the DBSCAN clustering algorithm. This includes the point itself. Basically, any set of pixels that can be considered as consisting of a polarity inversion line should have at least 1 core point. For the definition of core point, see the associated PIL module introduction notebook or  _sklearn.cluster.DBSCAN_.

__Attributes:__

__pos__\_ __threshold__: ___float___

The threshold for a positive pixel to be considered as a strong positive pixel.

__neg__\_ __threshold__: ___float___

The threshold for a negative pixel to be considered as a strong negative pixel.

**N**: ___Int___

The number of polarity inversion line segments founded.

__Methods__:

__.fit(_self,data,select=1,global_\_ _threshold=0.5,mode='image'_)__

Fit the PIL detection algorithm on input image or video.
 
__Parameters:__

 * __data__: ___Array of shape [height,width], or array of shape [video___\_ ___length,height,width]___.
   
   Input image data or video data. Missing values allowed. Any images with more than 20% missing values will not be processed, and any videos with more than 33% of missing values in the whole video will not be processed. Furthermore, an image with only 1 kind of polar will not be processed either.
   
   
 * __select__: ___Int, optional, default: 1___ 
 
   Maximum number of PIL segments to detect per image. 
   
   
 * __global__\_ __threshold__: ___float, optional, default: 0.5___
 
   The quantile threshold for defining a polarity inversion line as a strong polarity inversion line. All frames in the input video will have zero, one or multiple PIL segments detected (based on the _select_ parameter and the condition of each image), and all these PILs will be ranked according to their average gradient. All PIL segments above the quantile will be retained in the final output. 
   
   
 * __mode__: ___{"image","video"}___
 
   The scope in which the "strong" polar is defined. In the first step of the algorithm, the detector will search for the threshold of defining a pixel as a "strong" positive/negative pixel. When mode is "image", the pixel is strong as long as it is above the _polar_\_ _threshold_ quantile __within the image__. If the mode is "video", the pixel is strong only when it is above the _polar_\_ _threshold_ quantile __within the video__. So under the "video" mode, only the pixel that is strong bothspatially and temporally will be marked as "strong".
   

__Returns:__

If input is image data:

 * __PIL__\_ __segment__: ___dictionary, key is the label for PIL segment___.
 
   For an image which has *K* PIL segments detected, each PIL segment's information is stored in PIL_segment[i], _i=0,1,...,K-1_. And in PIL_segment[i], there are two keys, namely 'coor' and 'weight'. In PIL_segment[i]['coor'], one can access the coordinates of all points on the PIL segment. In PIL_segment[i]['weight'], one can obtain the average gradient for the PIL segment.
   
If input is video data:

 * __PIL__ \_ __video__: ___dictionary , key is the index for each frame___.
 
   For a video with K frames, each frame's PIL segments' information is stored in PIL_video[i], _i=0,1,...,K-1_. And for an arbitrary frame k, PIL_video[k] has the same dictionary structure as the output for frame k's image data. So to access the coordinate information for the i\_th PIL segment of frame *k* of the video, one can use PIL_video[k][0]['coor']. 

__.visualize(self,image,select=3,savefigure=False,figname='PIL')__

Fit the PIL on the image, and display the original image, the image with only the "strong" positive and negative pixels, and the image with the location of the detected polarity inversion line. This method is for the convenience of direct visual check when tuning the parameters for the PIL() class.

__Parameters:__

 * __image__: ___Array of shape [height,width]___.
   
   Input image, current version do not support video visualization yet.
   
   
 * __select__: ___Int, optional, default: 3___. 
 
   Maximum number of PIL segments to detect for the image. 
   
   
 * __savefigure__: ___Boolean, default: False___.
   
   A Boolean indicator on whether the output figure shall be saved to pdf format.
   
   
 * __figname__: ___String, default: 'PIL'___.
 
   The name of the saved figure if savefigure==True, the output name shall be "[figname].pdf".

### The parallelized implementation of the PIL detector class

_function_ __PIL__\_ __fit__ \_ __parallel__(file,mode='image',core=8,global_threshold=0.5,edge_retain=0.003,polar_threshold=0.90, radius=12,min_samples=5,select=1,window_size=5,filename='Nil')

Apply the PIL detector to a 3D-array video file or an hdf5 file containing all images of a single HARP region in a parallized way. Make sure multiprocessing is enabled before apply the program.

More introductions on this is available in the associated introduction notebook.

__Parameters__:


__file__: ___Array of shape [video___\_ ___length,height,width], or hdf5 file that has image stored in file['video0']['frameXXX']['channels'][:,:,2]___.

The input video upon which PIL for each frame is going to be detected. The input file can either be 3D numpy array or an hdf5 file.

__core__: ___Int, default: 8___.

Number of processors to parallelize the implementation.

__filename__: ___String, default: 'Nil'___.

The output file name. 

__All other parameters have the same definition as the documentation on `PIL()` class and the associated `.fit()` method.__

__Returns__:

If input is 3D numpy array:

 * __output_dict__: ___dictionary___. 
 
   The output filename shall be a pickled dictionary with name "[filename]PIL.pkl". The data structure of the output dictionary can be found in the returns PIL_video of the .fit() method of the PIL() class. 
   
   
If input is hdf5 file with the specified structure:

 * __output_file__: ___hdf5 file___.
 
   The output filename shall be a corresponding hdf5 file with name "[filename]PIL.hdf5". To access the information on the PIL segments of any frameXXX, after load the output file, use: `output_file['video0']['frameXXX']['PIL']`.